In [25]:
from dask.distributed import Client, Executor, progress

In [26]:
from dask import dataframe as dd

In [27]:
client = Client('10.10.100.7:8786')

In [28]:
client

<Client: scheduler='tcp://10.10.100.7:8786' processes=3 cores=18>

In [29]:
client.get_versions(check=True)

{'client': {'host': [('python', '3.6.1.final.0'),
   ('python-bits', 64),
   ('OS', 'Linux'),
   ('OS-release', '4.4.0-81-generic'),
   ('machine', 'x86_64'),
   ('processor', 'x86_64'),
   ('byteorder', 'little'),
   ('LC_ALL', 'None'),
   ('LANG', 'en_US.UTF-8'),
   ('LOCALE', 'en_US.UTF-8')],
  'packages': {'optional': [('numpy', '1.12.1'), ('pandas', '0.20.1')],
   'required': [('dask', '0.14.3'),
    ('distributed', '1.16.3'),
    ('msgpack', '0.4.8'),
    ('cloudpickle', '0.2.2'),
    ('toolz', '0.8.2')]}},
 'scheduler': {'host': [['python', '3.6.1.final.0'],
   ['python-bits', 64],
   ['OS', 'Linux'],
   ['OS-release', '4.4.0-81-generic'],
   ['machine', 'x86_64'],
   ['processor', 'x86_64'],
   ['byteorder', 'little'],
   ['LC_ALL', 'None'],
   ['LANG', 'en_US.UTF-8'],
   ['LOCALE', 'en_US.UTF-8']],
  'packages': {'optional': [['numpy', '1.12.1'], ['pandas', '0.20.1']],
   'required': [['dask', '0.14.3'],
    ['distributed', '1.16.3'],
    ['msgpack', '0.4.8'],
    ['cloudpickl

In [30]:
# create the dataframe by reading in the CSV
csv = '/root/stackoverflow.csv'

# load the csv file into a dask dataframe
df = dd.read_csv(csv)
df2= client.persist(df)
progress(df2)

In [31]:
df2.head()

,Unnamed: 0,id,owneruserid,posttypeid,creationdate,CreationDayOfWeek,acceptedanswerid,score,answercount,viewcount,...,CommentCount,FavoriteCount,site,acceptedAnswerId,acceptedAnswerCreationDate,acceptedAnswerCreationDayOfWeek,acceptedAnswerScore,acceptedAnswerCommentCount,acceptedAnswerOwnerUserId,site.1
0,0,4,8,1,2008-07-31 21:42:53,5,7.0,491,13,31416,...,4,37,stackoverflow,7.0,2008-07-31 22:17:58,5.0,349.0,1.0,9.0,stackoverflow
1,1,6,9,1,2008-07-31 22:08:09,5,31.0,217,5,14712,...,0,8,stackoverflow,31.0,2008-08-01 12:22:52,6.0,111.0,0.0,0.0,stackoverflow
2,2,9,1,1,2008-07-31 23:41:00,5,1404.0,1509,58,388201,...,6,341,stackoverflow,1404.0,2008-08-04 16:50:06,2.0,1403.0,30.0,212.0,stackoverflow
3,3,11,1,1,2008-07-31 23:55:38,5,1248.0,1182,33,120658,...,3,517,stackoverflow,1248.0,2008-08-04 13:57:26,2.0,792.0,16.0,268.0,stackoverflow
4,4,13,9,1,2008-08-01 00:42:39,6,NaN,482,24,138485,...,3,123,stackoverflow,12446.0,2008-08-15 16:27:32,6.0,64.0,1.0,5.0,stackoverflow


In [32]:
# count the number of total rows:
len(df2)

14015379

In [33]:
# list the columns
df2.columns

Index(['Unnamed: 0', 'id', 'owneruserid', 'posttypeid', 'creationdate',
       'CreationDayOfWeek', 'acceptedanswerid', 'score', 'answercount',
       'viewcount', 'owneruserid.1', 'lasteditoruserid', 'lasteditdate',
       'LastActivityDate', 'CommentCount', 'FavoriteCount', 'site',
       'acceptedAnswerId', 'acceptedAnswerCreationDate',
       'acceptedAnswerCreationDayOfWeek', 'acceptedAnswerScore',
       'acceptedAnswerCommentCount', 'acceptedAnswerOwnerUserId', 'site.1'],
      dtype='object')

In [34]:
# compute the average number of answers for each day of the week
df2.groupby(df2.CreationDayOfWeek).answercount.mean().compute()

CreationDayOfWeek
1    1.538740
2    1.564522
3    1.581523
4    1.579910
5    1.579372
6    1.590603
7    1.547162
Name: answercount, dtype: float64

In [35]:
df2.creationdate.head()

0    2008-07-31 21:42:53
1    2008-07-31 22:08:09
2    2008-07-31 23:41:00
3    2008-07-31 23:55:38
4    2008-08-01 00:42:39
Name: creationdate, dtype: object

In [36]:
import datetime
import pandas as pd

In [37]:
def to_datetime(row, col="creationdate"):
    try:
        return datetime.datetime.strptime(row[col], '%Y-%m-%d %H:%M:%S')
    except ValueError:
        # return a future date for easy discard
        return datetime.datetime.strptime('2020-01-01 12:00:00', '%Y-%m-%d %H:%M:%S')

def to_hour(row, col="creationDateTime"):
    # covert a datetime row to the hour
    return row[col].hour

def time_to_answer(row):
    try:
        td = datetime.datetime.strptime(row['acceptedAnswerCreationDate'], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(row['creationdate'], '%Y-%m-%d %H:%M:%S')
        return td.total_seconds()
    except (ValueError, TypeError):
        return None

# create a new column with datetime objects built from the creationdatetime field
df2["creationDateTime"] = df2.apply (lambda row: to_datetime(row), axis=1)

# create a row that holds the creation time hour
df2['creationHour'] = df2.apply(lambda row: to_hour(row), axis=1)

# create a row that holds the time to answer, in seconds
df2['timeToAnswer'] = df2.apply(lambda row: time_to_answer(row), axis=1)

client.persist(df2)
progress(df2)

/root/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py:2637: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [38]:
df2.columns

Index(['Unnamed: 0', 'id', 'owneruserid', 'posttypeid', 'creationdate',
       'CreationDayOfWeek', 'acceptedanswerid', 'score', 'answercount',
       'viewcount', 'owneruserid.1', 'lasteditoruserid', 'lasteditdate',
       'LastActivityDate', 'CommentCount', 'FavoriteCount', 'site',
       'acceptedAnswerId', 'acceptedAnswerCreationDate',
       'acceptedAnswerCreationDayOfWeek', 'acceptedAnswerScore',
       'acceptedAnswerCommentCount', 'acceptedAnswerOwnerUserId', 'site.1',
       'creationDateTime', 'creationHour', 'timeToAnswer'],
      dtype='object')

In [39]:
# count how many questions have an accepted answer:
df3 = df2[(df2.acceptedanswerid.notnull())]

client.persist(df3)
progress(df3)

In [40]:
len(df3)

7561435

In [42]:
df3.head()

,Unnamed: 0,id,owneruserid,posttypeid,creationdate,CreationDayOfWeek,acceptedanswerid,score,answercount,viewcount,...,acceptedAnswerId,acceptedAnswerCreationDate,acceptedAnswerCreationDayOfWeek,acceptedAnswerScore,acceptedAnswerCommentCount,acceptedAnswerOwnerUserId,site.1,creationDateTime,creationHour,timeToAnswer
0,0,4,8,1,2008-07-31 21:42:53,5,7.0,491,13,31416,...,7.0,2008-07-31 22:17:58,5.0,349.0,1.0,9.0,stackoverflow,2008-07-31 21:42:53,21,2105.0
1,1,6,9,1,2008-07-31 22:08:09,5,31.0,217,5,14712,...,31.0,2008-08-01 12:22:52,6.0,111.0,0.0,0.0,stackoverflow,2008-07-31 22:08:09,22,51283.0
2,2,9,1,1,2008-07-31 23:41:00,5,1404.0,1509,58,388201,...,1404.0,2008-08-04 16:50:06,2.0,1403.0,30.0,212.0,stackoverflow,2008-07-31 23:41:00,23,320946.0
3,3,11,1,1,2008-07-31 23:55:38,5,1248.0,1182,33,120658,...,1248.0,2008-08-04 13:57:26,2.0,792.0,16.0,268.0,stackoverflow,2008-07-31 23:55:38,23,309708.0
6,6,16,2,1,2008-08-01 04:59:34,6,12446.0,91,5,74191,...,531.0,2008-08-02 18:22:53,7.0,162.0,2.0,157.0,stackoverflow,2008-08-01 04:59:34,4,134599.0


In [41]:
# what percentage of questions that have an accepted answer got that answer within: 
#   i) one hour of the original post?
#  ii) one day of the original post?
# iii) one week of the original post?
#  iv) two weeks of the original post?
df_i = df3[(df3.timeToAnswer < 60*60)]
df_ii = df3[(df3.timeToAnswer < 60*60*24)]
df_iii = df3[(df3.timeToAnswer < 60*60*24*7)]
df_iv = df3[(df3.timeToAnswer < 60*60*24*7*2)]

ValueError: Metadata inference failed in `lt`.

Original error is below:
------------------------
TypeError("'<' not supported between instances of 'str' and 'int'",)

Traceback:
---------
  File "/root/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py", line 158, in raise_on_meta_error
    yield
  File "/root/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py", line 3087, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/root/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 861, in wrapper
    res = na_op(values, other)
  File "/root/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 765, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
  File "/root/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 745, in _comp_method_OBJECT_ARRAY
    result = lib.scalar_compare(x, y, op)
  File "pandas/_libs/lib.pyx", line 769, in pandas._libs.lib.scalar_compare (pandas/_libs/lib.c